In [1014]:
import numpy as np
from tqdm import tqdm
from time import sleep

In [1015]:
class NN():

    def __init__(self, hiddenLayer_neuron = 2,learning_rate = 0.1, batch_size = 2):

        self.data = np.array([[0,0],
                              [0,1],
                              [1,0],
                              [1,1]])
        self.target = np.array([[0],[1],[1],[0]])

        self.hl_neuron = hiddenLayer_neuron

        
        self.w_hidden = np.random.uniform(size=(2,self.hl_neuron))
        self.w_out = np.random.uniform(size=(self.hl_neuron,1))

        self.b_hidden = np.random.uniform(size=(1,self.hl_neuron))
        self.b_out = np.random.uniform(size=(1,1))

        self.z0s = []
        self.z1s = []

        self.batch_size = batch_size

        self.lr = learning_rate

        print("Init: ")
        self.print_weights()

    def ReLU(self,x):
        return x * (x > 0) 
    
    def bcel(self, y, y_hat):
        return -(y*np.log(y_hat) + (1-y) * np.log(1-y_hat))
    
    def drv_loss_start(self, y, y_hat):
        return 1 / (1-y_hat) if y == 0 else -1 / y_hat

    def drv_loss(self, y, y_hat):
        return y_hat-y
    
    def drv_sigmoid(self, z):
        return self.sigmoid(z) * (1-self.sigmoid(z))
    
    '''def drv_sigmoid(self, sigmoid_value):
        return sigmoid_value * (1-sigmoid_value)'''
    
    def drv_ReLU(self, z):
        return 1 * (z > 0)

    def sigmoid(self,x):
        return 1/(1 + np.exp(-x))
    
    def execute(self,x):
        result = self.sigmoid(self.forward_pass(x)[0])
        return np.round(result)
    
    def forward_pass(self,x):
        x = np.array(x)
        z0 = np.dot(x,self.w_hidden) + self.b_hidden
        hidden = self.ReLU(z0)
        z1 = np.dot(hidden,self.w_out) + self.b_out
        return z1, z0
    

    def backward_pass(self,x,y,z1,z0):

        x = np.array(x)

        y_hat = self.sigmoid(z1)
        
        d1 = self.drv_loss(y,y_hat) / self.batch_size
        d0 = self.drv_loss(y,y_hat) * np.dot(self.drv_ReLU(z0),self.w_out) / self.batch_size

        self.w_out -= self.lr * np.dot(self.ReLU(z0).T,d1) 
        self.w_hidden -=  self.lr * np.dot(x.T,d0)

        self.b_out -= self.lr * np.sum(d1, axis=0, keepdims=True)
        self.b_hidden -= self.lr * np.sum(d0, axis=0, keepdims=True)
        
    def print_weights(self):
        print("W_hidden: " + str(self.w_hidden)) 
        print("B_hidden: " + str(self.b_hidden)) 
        print("W_out: " + str(self.w_out)) 
        print("B_out: " + str(self.b_out), end="\n\n")


    def accuracy(self):
        counter = 0
    
        for i in range(len(self.data)):
            if(self.execute(self.data[i]) == self.target[i]):
                counter += 1
        loss = 0
        for i in range(len(self.data)):
            loss += self.bcel(self.target[i],self.sigmoid(self.forward_pass(self.data[i])[0]))

        return counter / len(self.data), loss / len(self.data)
     
    def train(self,epochs, fast_train = False):


        if(epochs > 0):
            for epoch in range(epochs):
                print("Starting epoch: " + str(epoch+1))
                index = index = np.random.randint(4,size=self.batch_size)
                batch = []
                y = []
                for i in index:
                    batch.append(self.data[i])
                    y.append(self.target[i])

                z1,z0 = self.forward_pass(batch)
                self.backward_pass(batch,y,z1,z0)

                acc, loss =  self.accuracy()
                print("Current accuracy: " +f"{acc*100}%")
                print("Current loss: " +f"{loss}")
        else:
            epoch = 1
            acc = 0
            while acc != 1:
                print("Starting epoch: " + str(epoch))
                index = index = np.random.randint(4,size=self.batch_size)
                batch = []
                y = []
                for i in index:
                    batch.append(self.data[i])
                    y.append(self.target[i])

                z1,z0 = self.forward_pass(batch)
                self.backward_pass(batch,y,z1,z0)

                acc, loss =  self.accuracy()
                print("Current accuracy: " +f"{acc*100}%")
                print("Current loss: " +f"{loss}")

                epoch +=1 

        
        self.print_weights
                
        '''if epochs > 0:
            for epoch in range(epochs):
                index = np.random.randint(4,size=self.batch_size)
                for i in index:
                    z1, z0 = self.forward_pass(self.data[i])
                    self.z0s.append(z0)
                    self.z1s.append(z1)

                self.z1s.reverse()
                self.z0s.reverse()

                if not fast_train:

                    for i in tqdm(index,desc="Epoch: " + str(epoch)): 
                        self.backward_pass(self.data[i],self.target[i])
                        sleep(.1)
                    acc, loss =  self.accuracy()
                    print("Current accuracy: " +f"{acc*100}%")
                    print("Current loss: " +f"{loss}")
                
                else:
                    for i in index:
                        self.backward_pass(self.data[i],self.target[i])
                    acc, loss =  self.accuracy()
            
            print("Current accuracy: " +f"{acc*100}%")
            print("Current loss: " +f"{loss}")

        
        else:
            epoch = 0
            acc = 0
            while acc  != 1:
                index = np.random.randint(4,size=self.batch_size)
                for i in index:
                    z1, z0 = self.forward_pass(self.data[i])
                    self.z0s.append(z0)
                    self.z1s.append(z1)

                self.z1s.reverse()
                self.z0s.reverse()

                if not fast_train:

                    for i in tqdm(index,desc="Epoch: " + str(epoch)): 
                        self.backward_pass(self.data[i],self.target[i])
                        sleep(.1)
                    acc, loss =  self.accuracy()
                    print("Current accuracy: " +f"{acc*100}%")
                    print("Current loss: " +f"{loss}")
                
                else:
                    for i in index:
                        self.backward_pass(self.data[i],self.target[i])
                    acc, loss =  self.accuracy()
                    print("Current accuracy: " +f"{acc*100}%")
                    print("Current loss: " +f"{loss}")
                    
                epoch += 1
        print("Epochs trained: " + str(epoch+1))
        self.print_weights()
        print(acc)
        return acc       '''

In [1016]:
nn = NN(8,learning_rate=0.1, batch_size= 200)

Init: 
W_hidden: [[0.52665789 0.09348639 0.06644959 0.46357852 0.92379167 0.6757116
  0.86593745 0.83249435]
 [0.71183557 0.99722739 0.76902738 0.3237747  0.11759066 0.30603526
  0.24185284 0.28344915]]
B_hidden: [[0.08695019 0.97409431 0.99023353 0.52824938 0.85903529 0.69129579
  0.13538677 0.34027063]]
W_out: [[0.30281776]
 [0.99435593]
 [0.95250856]
 [0.27453381]
 [0.89150473]
 [0.70585148]
 [0.23770862]
 [0.04090144]]
B_out: [[0.64995528]]



In [1017]:
nn.train(epochs = -1,fast_train=True)

Starting epoch: 1
Current accuracy: 50.0%
Current loss: [[2.27756061]]
Starting epoch: 2
Current accuracy: 50.0%
Current loss: [[1.64759333]]
Starting epoch: 3
Current accuracy: 50.0%
Current loss: [[1.22617123]]
Starting epoch: 4
Current accuracy: 50.0%
Current loss: [[0.94766591]]
Starting epoch: 5
Current accuracy: 50.0%
Current loss: [[0.79085967]]
Starting epoch: 6
Current accuracy: 50.0%
Current loss: [[0.6992188]]
Starting epoch: 7
Current accuracy: 50.0%
Current loss: [[0.66562902]]
Starting epoch: 8
Current accuracy: 50.0%
Current loss: [[0.65729931]]
Starting epoch: 9
Current accuracy: 50.0%
Current loss: [[0.65118704]]
Starting epoch: 10
Current accuracy: 50.0%
Current loss: [[0.64641034]]
Starting epoch: 11
Current accuracy: 50.0%
Current loss: [[0.64435803]]
Starting epoch: 12
Current accuracy: 50.0%
Current loss: [[0.64168321]]
Starting epoch: 13
Current accuracy: 50.0%
Current loss: [[0.63996715]]
Starting epoch: 14
Current accuracy: 50.0%
Current loss: [[0.63715979]]
St

In [1018]:
print(nn.execute([0,0]))
print(nn.execute([0,1]))
print(nn.execute([1,0]))
print(nn.execute([1,1]))

[[0.]]
[[1.]]
[[1.]]
[[0.]]
